In [1]:
import numba

In [2]:
from numba import jit
import numpy as np

In [3]:
# test numba in python code 
x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


In [4]:
def calc_energy(Img):
    I = Img.astype(float)
    I = np.concatenate((I,np.zeros((I.shape[0],1))), axis=1)
    I = np.concatenate((I,np.zeros((1,I.shape[1]))), axis=0)
    Dx = I[:-1, 1:] - I[:-1, :-1]
    Dy = I[1:, :-1] - I[:-1, :-1]
    E = np.sqrt(Dx**2 + Dy**2)
    E = E.astype(np.uint8)
    return E
    

In [5]:
def minimum_seam(img):
    r, c = img.shape
    energy_map = calc_energy(img)
    M = energy_map.copy()
    backtrack = np.zeros_like(M, dtype=np.int)
    for i in range(1, r):
        for j in range(0, c):
            # Handle the left edge of the image, to ensure we don't index -1
            if j == 0:
                idx = np.argmin(M[i - 1, j:j + 2])
                backtrack[i, j] = idx + j
                min_energy = M[i - 1, idx + j]
            else:
                idx = np.argmin(M[i - 1, j - 1:j + 2])
                backtrack[i, j] = idx + j - 1
                min_energy = M[i - 1, idx + j - 1]

            M[i, j] += min_energy

    return M, backtrack


In [6]:
def carve_column(img):
    r, c = img.shape
    M, backtrack = minimum_seam(img)
    mask = np.ones((r, c), dtype=np.bool)
    j = np.argmin(M[-1])
    for i in reversed(range(r)):
        mask[i, j] = False
        j = backtrack[i, j]
    mask = np.stack([mask] * 3, axis=2)
    img = cv2.resize(img,(int(r),int(c-1)))
    return img

def crop_c(img, scale_c):
    r, c = img.shape
    new_c = int(scale_c * c)
    for i in range(c - new_c): # use range if you don't want to use tqdm
        print("processing ...: " + i)
        img = carve_column(img)
    return img


In [ ]:
    in_filename = './images/pietro_carved.jpg'
    out_filename = './images/out.jpg'
    # Buoc 1: Đọc ảnh mức xám Img
    # cv2.imread('toctien.jpg')
    Img = cv2.imread('./images/pietro_carved.jpg', 0)
    # cv2.imshow('Anh goc', Img)
    print('Kich thuoc ban dau cua Img: ', Img.shape)
    E = calc_energy(Img)
    # Img = carve_column(Img)
    Img = crop_c(Img, 0.5)
    print(Img)
    cv2.imwrite(out_filename, Img)
    # print('Kich thuoc anh nang luong E: ', E.shape)
    # cv2.imshow('Anh nang luong', E)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()